In [1]:
import pandas
import data_process
import tools
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "d:\Anaconda\envs\DL\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "d:\Anaconda\envs\DL\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "d:\Anaconda\envs\DL\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\Anaconda\envs\DL\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "d:\Anaconda\envs\DL\lib\site-packages\ipykernel\kernelapp

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "d:\Anaconda\envs\DL\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "d:\Anaconda\envs\DL\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "d:\Anaconda\envs\DL\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\Anaconda\envs\DL\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "d:\Anaconda\envs\DL\lib\site-packages\ipykernel\kernelapp

AttributeError: _ARRAY_API not found

In [2]:
class rnn(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, hidden_size, device):
        super(rnn, self).__init__()
        self.device = device
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 2)  # 假设是二分类任务
    
    def forward(self, x):
        x = self.embedding(x)  # 获取词嵌入
        out, _ = self.rnn(x)  # RNN 前向传播
        out = out[:, -1, :]  # 取最后一个时间步的输出
        out = self.fc(out)  # 输出层
        return out

In [3]:

class OptimizedRNN(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, hidden_size, device):
        super(OptimizedRNN, self).__init__()
        self.device = device
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_size, batch_first=True, bidirectional=True)  # 双向LSTM
        self.dropout = nn.Dropout(0.5)  # Dropout正则化
        self.fc = nn.Linear(hidden_size * 2, 2)  # 双向LSTM输出维度需要乘2
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.embedding(x)  # 获取词嵌入
        out, _ = self.rnn(x)  # LSTM前向传播
        out = out[:, -1, :]  # 取最后一个时间步的输出
        out = self.dropout(out)  # 应用Dropout
        out = self.fc(out)  # 输出层
        out = self.softmax(out)  # 进行Softmax归一化
        return out



In [4]:


class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx,device):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)


包装数据

In [5]:
data  = pandas.read_csv('./motionClassify.csv')
vocab = data_process.gen_vocab(data)
data_train  =  data_process.gen_dataset(data[:40000],vocab)
data_test = data_process.gen_dataset(data[40000:],vocab)
Batch_size = 64
train_iter = torch.utils.data.DataLoader(data_train,Batch_size,shuffle=True)
test_iter = torch.utils.data.DataLoader(data_test,Batch_size,shuffle=True)

In [6]:
def get_pad_index(vocab):
    # 如果 vocab 中没有 <pad>，手动添加一个索引
    if '<pad>' not in vocab:
        # 假设 <pad> 的索引为词汇表当前大小
        PAD_IDX = len(vocab)
    else:
        # 如果 vocab 中已经有 <pad>，直接获取它的索引
        PAD_IDX = vocab['<pad>']
    
    print(f'PAD index: {PAD_IDX}')
    return PAD_IDX


In [9]:
lr=0.05
PAD_IDX = get_pad_index(vocab)
criterion = torch.nn.CrossEntropyLoss()
device = torch.device('cpu' if not torch.cuda.is_available() else 'cuda:0')
#net1 = rnn(num_embeddings=len(vocab),embedding_dim=256,hidden_size=256,device=device)
#net2 = OptimizedRNN(num_embeddings=len(vocab), embedding_dim=256, hidden_size=256, device=device)
net3=RNN(vocab_size=len(vocab),embedding_dim=100,hidden_dim=256,output_dim=1,n_layers=2,bidirectional=True,dropout=0.5,pad_idx=vocab.token_to_idx['<pad>'],device=device)
#optimizer1 = torch.optim.SGD(net1.parameters(),lr)
#optimizer2 = torch.optim.SGD(net2.parameters(),lr)
optimizer = torch.optim.Adam(net3.parameters())



KeyboardInterrupt: 

In [11]:
tools.train(net1,train_iter,device,optimizer1,criterion)

 16%|█▌        | 100/625 [02:48<15:01,  1.72s/it]

batch100,loss = 0.681061327457428


 32%|███▏      | 200/625 [14:15<1:01:03,  8.62s/it]

batch200,loss = 0.6917207837104797


 48%|████▊     | 300/625 [20:43<09:09,  1.69s/it]  

...

 80%|████████  | 500/625 [30:45<03:32,  1.70s/it]

batch500,loss = 0.6930347681045532


 96%|█████████▌| 600/625 [35:53<00:38,  1.55s/it]

batch600,loss = 0.6938478946685791


100%|██████████| 625/625 [36:31<00:00,  3.51s/it]


In [12]:
tools.test(net1,test_iter,device)

100%|██████████| 157/157 [00:12<00:00, 12.75it/s]

accuracy = 0.49959999322891235


准确率50%，接近自然概率，训练没有效果

In [8]:
tools.train(net2,train_iter,device,optimizer2,criterion)

 16%|█▌        | 100/625 [18:39<1:04:42,  7.39s/it]

batch100,loss = 0.6843875050544739


 32%|███▏      | 200/625 [33:56<50:37,  7.15s/it]  

batch200,loss = 0.6883144378662109


 48%|████▊     | 297/625 [54:37<1:00:19, 11.03s/it]


KeyboardInterrupt: 

In [ ]:
tools.test(net2,test_iter,device)

In [ ]:
#tools.train(net3,train_iter,device,optimizer,criterion)

In [ ]:
#tools.test(net3,test_iter,device)